In [33]:
import os
import json
import random
import logging
# import pandas as pd
import spacy
from tqdm import tqdm
from spacy.tokens import DocBin

VISUALIZATION_STATUS = True

In [34]:
def convert_spacy_format(data, output_path="data/unkonwn.spacy"):
    nlp = spacy.blank("bn") # load a new spacy model
    db = DocBin() # create a DocBin object
    c = 0
    for text, annot in tqdm(data): # data in previous format
        doc = nlp.make_doc(text) # create doc object from text
        ents = []
        for start, end, label in annot["entities"]: # add character indexes
            
            span = doc.char_span(start, end, label=label, alignment_mode="strict")
            # print(start, end, label, span)
            if span is None:
                s = doc.text
                sub_E = s[end:]
                sub_S = s[:start]
                end = end+ (0 if len(sub_E.split(" ", 1)[0]) <= 0 else len(sub_E.split(" ", 1)[0]))
                start = start - (0 if len(sub_S.rsplit(" ", 1)[-1]) <= 0 else len(sub_S.rsplit(" ", 1)[-1]))
                
                span = doc.char_span(start, end, label=label, alignment_mode="strict")
                # print("Text:", text)
                # print("============:", s[start:end], "span:", span, label)
                if span is None:
                    print("++++++++++++++++++++++++++++Skipping entity Start++++++++++++++++++++++++++++")
                    print(start, end, label, span)
                    print(doc.text[start:end],doc.text[start],doc.text[end-1],'kh',sep='|')
                    print("++++++++++++++++++++++++++++Skipping entity End++++++++++++++++++++++++++++++")
                    break
                    c+=1
            else:
                ents.append(span)
            doc.ents = ents # label the text with the ents
        if VISUALIZATION_STATUS:
            spacy.displacy.render(doc, style="ent", jupyter=True)
        db.add(doc)
    db.to_disk(output_path) # save the docbin object

In [35]:
def convert_jsonl_to_spacy(doccano_JSON_FilePath):
    # try:
    training_data = []
    lines=[]
    with open(doccano_JSON_FilePath, 'r') as f:
        lines = f.readlines()

    for line in lines:
        data = json.loads(line)
        # print(data)
        text = data['text']
        entities = data['label']
        if len(entities)>0:
            training_data.append((text, {"entities" : entities}))
    return training_data

In [36]:
def save_jsonl(data, filename):
    with open(filename, 'w', encoding='utf-8') as file:
        for item in data:
            json.dump(item, file, ensure_ascii=False)
            file.write('\n')

In [37]:
train_file = "../data/bangla_ner_span_based/train.jsonl"
val_file  = ""
output_dir = "../data/bangla_ner_span_based"

training_data = convert_jsonl_to_spacy(train_file)

print("Training Data:",len(training_data))
# print("Validation Data:",len(validation_data))
# save_jsonl(training_data, os.path.join(output_dir,"val.jsonl"))

TypeError: list indices must be integers or slices, not str

In [26]:
convert_spacy_format(
    training_data, 
    output_path=os.path.join(output_dir,"train.spacy")
)
# convert_spacy_format(validation_data, output_path=os.path.join(output_dir,"val.spacy"))

  0%|                                                                                                                          | 0/4515 [00:00<?, ?it/s]

  2%|██▍                                                                                                            | 100/4515 [00:00<00:04, 996.32it/s]

  3%|███                                                                                                            | 125/4515 [00:00<00:04, 998.09it/s]


ValueError: [E1010] Unable to set entity information for token 47 which is included in more than one span in entities, blocked, missing or outside.